### **1.2. Implementación e Interpretación de Lasso y Ridge**

In [41]:
# importamos librerias necesarias
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', message="Coordinate descent without L1 regularization may lead to unexpected results and is discouraged.")
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [50]:
# Basic
import numpy as np
import pandas as pd
from functools import partial

# Scikit-Learn
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV

# Matplotlib
from matplotlib.pyplot import subplots

**a) Lasso: Entrene un objeto LassoCV con 10 pliegues (cv=10) sobre los datos de entrenamiento para encontrar el lambda optimo. Imprima el valor encontrado y grafique**

In [ ]:
lasso_cv = LassoCV(cv=10, random_state=42)

lasso_cv.fit(X_train_scaled, Y_train)

# Calculamos el promedio de esos 10 errores para cada valor de alfa
mse_promedio = lasso_cv.mse_path_.mean(axis=1)

# Calculamos el logaritmo 
log_alphas = np.log10(lasso_cv.alphas_)

plt.figure(figsize=(10, 6))
plt.plot(log_alphas, mse_promedio)


plt.axvline(np.log10(lasso_cv.alpha_), color='red', linestyle='--',
            label=f'Log(Lamda) óptimo: {np.log10(lasso_cv.alpha_):.4f}')

plt.xlabel(r'$\log(\lambda)$')
plt.ylabel('Error Cuadrático Medio (MSE)')
plt.title('Validación Cruzada de Lasso: MSE vs Lambda')
plt.legend()
plt.show()

**b) Muestre los coeficientes del mejor modelo Lasso. En este dataset, ¿Lasso elimino
alguna variable (establecio su coeficiente en cero)? Basado en los coeficientes no nulos, ¿cuáles parecen ser los predictores mas importantes segun el modelo?**

In [ ]:
# Print the optimal lambda
# Extraer coeficientes del objeto lasso_cv que ya entrenamos
lasso_coefs = lasso_cv.coef_

# Crear el DataFrame 
lasso_df = pd.DataFrame({'Variable': X.columns, 'Coeficiente': lasso_coefs})

# Se ordena según el coeficiente
lasso_df = lasso_df.sort_values(by='Coeficiente', ascending=False)

# Resultado
lasso_df

**c) Ridge: De manera similar, entrene un objeto RidgeCV con 10 pliegues para encontrar el lambda ´optimo. Imprima el valor encontrado y grafique**

In [ ]:
alphas = np.logspace(-3, 8, 100)
ridge_cv = RidgeCV(alphas=alphas, cv=10)

ridge_cv.fit(X_train_scaled, Y_train)


coefs = []

for a in alphas:
    # Usamos Ridge normal para la trayectoria, ya que solo queremos ajustar un alfa a la vez
    from sklearn.linear_model import Ridge
    ridge_tmp = Ridge(alpha=a)
    ridge_tmp.fit(X_train_scaled, Y_train)
    coefs.append(ridge_tmp.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)


plt.xscale("log")
plt.xlabel(r"$\lambda$")
plt.ylabel("Valor de los Coeficientes")
plt.title("Ridge: Trayectoria de Coeficientes")
plt.grid(True, alpha=0.3)
plt.legend(X.columns, loc='upper right', bbox_to_anchor=(1.3, 1)) # Lista de variables
plt.show()

**d) Muestre los coeficientes del mejor modelo Ridge. Compare las magnitudes de los
coeficientes de Ridge con los de Lasso. ¿Ridge establece alúun coeficiente exactamente en cero? Explique por qée esta es una diferencia entre ambos éetodos d 
regularizaó´on (L1 vs. L2**).


In [ ]:
#Graph
ridge_fig, ax = subplots(figsize=(8,6))
ax.errorbar(np.log(lambdas), -grid.cv_results_['mean_test_score'], yerr=grid.cv_results_['std_test_score'] / np.sqrt(K))
ax.axvline(np.log(best_alpha_mse), color='k', linestyle='--')
ax.set_xlabel(r'$\log(\lambda)$', fontsize=15)
ax.set_ylabel('Cross-validated MSE', fontsize=15)
ax.legend(['Optimal $\\lambda$', 'MSE per $\\lambda$'])

best_alpha_mse = grid.best_params_['ridge__alpha']

# Print best lambda
print(f"The best value of lambda (alpha) found using MSE is: {best_alpha_mse}")
print(f"The log of this lambda is: {np.log(best_alpha_mse)}")

**e) Evaluacion Final: Evalúe el rendimiento de ambos modelos finales (Lasso optimo y
Ridge optimo) sobre el conjunto de prueba. Calcule y reporte el Error Cuadratico
Medio (MSE) para cada uno. ¿Qué modelo tuvo un mejor rendimiento predictivo
en este caso?**